In [49]:
import os
import cv2
import csv
import numpy as np

# --- PARAMETERS ---
IMG_DIR   = '.'  # folder with your dino*.png
GOOD_FILE = 'dino_good_silhouette_images.txt'
ANG_FILE  = 'dino_ang.txt'
PAR_FILE  = 'dino_par.txt'

RATIO_THR   = 0.7   # Lowe’s ratio test threshold
RANSAC_THR  = 1.0   # RANSAC reprojection threshold (pixels)
MIN_MATCHES = 8     # minimum matches after ratio test

# --- 1. Load good‐silhouette list ---
with open(GOOD_FILE, 'r') as f:
    good_imgs = {ln.strip() for ln in f if ln.strip()}

# --- 2. Load angles and keep only good images ---
ang_map = {}
with open(ANG_FILE, 'r') as f:
    for ln in f:
        parts = ln.strip().split()
        if len(parts) != 3:
            continue
        try:
            lat, lon = float(parts[0]), float(parts[1])
        except ValueError:
            continue
        name = parts[2]
        if name in good_imgs:
            ang_map[name] = (lat, lon)

# sort by latitude, then longitude
sorted_imgs = sorted(ang_map.keys(), key=lambda n: (ang_map[n][0], ang_map[n][1]))

# --- 3. Parse calibration file to extract K ---
K = None
with open(PAR_FILE, 'r') as f:
    for ln in f:
        parts = ln.strip().split()
        # each line: "dino0001.png fx cx  fy cy  ...", adjust as needed
        if len(parts) >= 7 and parts[0].endswith('.png'):
            fx, cx = float(parts[1]), float(parts[3])
            fy, cy = float(parts[5]), float(parts[6])
            K = np.array([[fx,   0, cx],
                          [  0, fy, cy],
                          [  0,   0,  1]])
            break
if K is None:
    raise RuntimeError(f"Could not parse intrinsics from {PAR_FILE}")

# prepare SIFT and BFMatcher
sift = cv2.SIFT_create()
bf   = cv2.BFMatcher(cv2.NORM_L2)

# make output folders
os.makedirs('matches', exist_ok=True)

# CSV log
with open('pair_results.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['img1','img2','raw_matches','inliers','t_x','t_y','t_z'])

    # --- 4. Loop over consecutive pairs ---
    for i in range(len(sorted_imgs) - 1):
        img1_name = sorted_imgs[i]
        img2_name = sorted_imgs[i+1]

        # load grayscale
        img1 = cv2.imread(os.path.join(IMG_DIR, img1_name), cv2.IMREAD_GRAYSCALE)
        img2 = cv2.imread(os.path.join(IMG_DIR, img2_name), cv2.IMREAD_GRAYSCALE)
        if img1 is None or img2 is None:
            print(f"Warning: couldn’t load {img1_name} or {img2_name}, skipping")
            continue

        # detect & describe
        kp1, des1 = sift.detectAndCompute(img1, None)
        kp2, des2 = sift.detectAndCompute(img2, None)

        # raw knn matches + Lowe’s ratio test
        raw = bf.knnMatch(des1, des2, k=2)
        good = []
        pts1, pts2 = [], []
        for m, n in raw:
            if m.distance < RATIO_THR * n.distance:
                good.append(m)
                pts1.append(kp1[m.queryIdx].pt)
                pts2.append(kp2[m.trainIdx].pt)

        pts1 = np.float32(pts1)
        pts2 = np.float32(pts2)
        raw_count = len(good)
        if raw_count < MIN_MATCHES:
            print(f"{img1_name}–{img2_name}: only {raw_count} after ratio test; skipping")
            continue

        # estimate Fundamental matrix via RANSAC
        F, mask = cv2.findFundamentalMat(pts1, pts2,
                                         cv2.FM_RANSAC,
                                         ransacReprojThreshold=RANSAC_THR,
                                         confidence=0.99)
        if F is None:
            print(f"{img1_name}–{img2_name}: Fundamental matrix failed; skipping")
            continue

        inlier_mask = mask.ravel().astype(bool)
        inlier_count = int(inlier_mask.sum())

        # --- 5. Compute Essential & recover pose ---
        E = K.T @ F @ K
        pts1_in = pts1[inlier_mask]
        pts2_in = pts2[inlier_mask]
        _, R, t, _ = cv2.recoverPose(E, pts1_in, pts2_in, K)

        # --- 6. Draw & save inlier matches ---
        mask_ints = [int(m) for m in inlier_mask]  # cv2 wants ints, not bools
        vis = cv2.drawMatches(
            img1, kp1, img2, kp2, good, None,
            matchColor=(0,255,0),
            singlePointColor=None,
            matchesMask=mask_ints,
            flags=cv2.DrawMatchesFlags_DEFAULT,
            matchesThickness=1
        )
        out_fn = f"matches/{img1_name[:-4]}_{img2_name[:-4]}_inliers.png"
        cv2.imwrite(out_fn, vis)

        # log to CSV
        tx, ty, tz = t.ravel()
        writer.writerow([img1_name, img2_name,
                         raw_count, inlier_count,
                         tx, ty, tz])

        print(f"{img1_name}–{img2_name}: {inlier_count}/{raw_count} inliers, t = [{tx:.3f}, {ty:.3f}, {tz:.3f}]")

print("Done.  Inlier visuals in ‘matches/’, summary in ‘pair_results.csv’.")


dino0049.png–dino0048.png: 61/83 inliers, t = [0.020, -0.471, -0.882]
dino0048.png–dino0047.png: 54/82 inliers, t = [0.059, -0.864, 0.500]
dino0047.png–dino0046.png: 45/77 inliers, t = [-0.005, -0.534, -0.846]
dino0046.png–dino0045.png: 34/55 inliers, t = [0.035, -0.339, 0.940]
dino0045.png–dino0044.png: 25/45 inliers, t = [0.048, 0.091, 0.995]
dino0044.png–dino0043.png: 21/33 inliers, t = [-0.064, 0.381, 0.922]
dino0043.png–dino0042.png: 13/22 inliers, t = [-0.002, -0.036, 0.999]
dino0042.png–dino0041.png: 14/20 inliers, t = [-0.050, 0.039, 0.998]
dino0041.png–dino0040.png: 14/23 inliers, t = [0.064, -0.224, -0.972]
dino0040.png–dino0039.png: 21/34 inliers, t = [0.041, -0.464, -0.885]
dino0039.png–dino0038.png: 26/47 inliers, t = [0.032, -0.220, -0.975]
dino0038.png–dino0037.png: 45/74 inliers, t = [0.020, -0.414, -0.910]
dino0037.png–dino0036.png: 53/81 inliers, t = [0.024, -0.326, -0.945]
dino0036.png–dino0035.png: 71/104 inliers, t = [0.019, -0.433, -0.901]
dino0035.png–dino0034.pn

In [50]:
# ── cell: Initial 3D Reconstruction via E decomposition ──

import numpy as np
import cv2

# assume these are already in your notebook:
#   K           : (3×3) calibration matrix
#   F           : (3×3) fundamental matrix from findFundamentalMat()
#   pts1, pts2  : (N×2) all matched points before RANSAC
#   mask        : (N×1) inlier mask (0/1) from findFundamentalMat

# 1) Extract only the inlier correspondences
mask_bool = mask.ravel().astype(bool)
pts1_in = pts1[mask_bool]
pts2_in = pts2[mask_bool]

# 2) Compute and “fix” the Essential matrix
E = K.T @ F @ K
U, S, Vt = np.linalg.svd(E)
# enforce (σ,σ,0) singular values
sigma = (S[0] + S[1]) / 2.0
E = U @ np.diag([sigma, sigma, 0]) @ Vt

# 3) Build the four (R, t) hypotheses
W = np.array([[0, -1,  0],
              [1,  0,  0],
              [0,  0,  1]])
R1 = U @ W @ Vt
R2 = U @ W.T @ Vt
t0 = U[:, 2]

# ensure det(R)=+1
if np.linalg.det(R1) < 0: R1 *= -1
if np.linalg.det(R2) < 0: R2 *= -1

candidates = [
    (R1,  t0),
    (R1, -t0),
    (R2,  t0),
    (R2, -t0),
]

# helper: triangulate and count positive‐depth points
def count_in_front(R, t, K, pts1, pts2):
    # Projection matrices
    P1 = K @ np.hstack((np.eye(3), np.zeros((3,1))))
    P2 = K @ np.hstack((R, t.reshape(3,1)))
    # need points in normalized coordinates (undistorted)
    p1n = cv2.undistortPoints(pts1.reshape(-1,1,2), K, None)
    p2n = cv2.undistortPoints(pts2.reshape(-1,1,2), K, None)
    # triangulate (4×N)
    X4 = cv2.triangulatePoints(P1, P2, p1n, p2n)
    X3 = (X4[:3] / X4[3]).T  # N×3
    # depths in cam1 = Z, in cam2 = third coord of R·X + t
    z1 =  X3[:, 2]
    z2 = (R @ X3.T + t.reshape(3,1))[2]
    return np.sum((z1 > 0) & (z2 > 0)), X4

# 4) Pick the best pose
best_count = 0
for R_c, t_c in candidates:
    cnt, X4 = count_in_front(R_c, t_c, K, pts1_in, pts2_in)
    if cnt > best_count:
        best_count = cnt
        R_final, t_final, X4_final = R_c, t_c, X4

print(f"→ picked pose with {best_count} points in front.")
print("Rotation R_final:\n", R_final)
print("Translation t_final:\n", t_final)

# 5) (optional) convert to Euclidean 3D
X3D = (X4_final[:3] / X4_final[3]).T  # shape: (best_count×3)
print("First few 3D points:\n", X3D[:5])


→ picked pose with 84 points in front.
Rotation R_final:
 [[ 0.99672795 -0.08008046  0.01097805]
 [ 0.07973585  0.99639762  0.02887849]
 [-0.01325111 -0.02790866  0.99952264]]
Translation t_final:
 [-0.00692055 -0.49965152 -0.86619886]
First few 3D points:
 [[-2.376378  -1.3716363 23.33098  ]
 [-2.376358  -1.3714566 23.331608 ]
 [-2.3762972 -1.3714077 23.331572 ]
 [-2.3762898 -1.3713775 23.3317   ]
 [-2.3761408 -1.3717183 23.330488 ]]


In [54]:
import os
import cv2
import numpy as np

# Initialize counters
skipped_count = 0
failed_count = 0
working_count = 0

# 0) Reconstruct inlier_matches from your initial pair
#    (good = list of DMatch, mask = Nx1 mask from findFundamentalMat)
mask_bool = mask.ravel().astype(bool)
inlier_matches = [m for m, valid in zip(good, mask_bool) if valid]

# 1) Convert homogeneous 4D points → Euclidean 3D
X3D = (X4_final[:3] / X4_final[3]).T  # shape: (M,3)

# 2) Build a map: kp1 index → index in X3D
kp1_to_3d = {m.queryIdx: i for i, m in enumerate(inlier_matches)}

# Prepare storage
R_all, t_all = [], []

# Set up SIFT with a higher number of features or switch to ORB for speed
sift = cv2.SIFT_create(nfeatures=1000)  # Increase features detected

# Use FLANN-based Matcher with cross-checking for better matches
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)  # Increase checks for better match quality
flann = cv2.FlannBasedMatcher(index_params, search_params)

# Iterate through the images
for img_name in sorted_imgs[2:]:
    # Load new image & compute features
    img = cv2.imread(os.path.join(IMG_DIR, img_name), cv2.IMREAD_GRAYSCALE)
    kp2, des2 = sift.detectAndCompute(img, None)

    # Match back to the first image's descriptors using FLANN
    raw2 = flann.knnMatch(des2, des1, k=2)

    # Apply ratio test to remove bad matches (adjusting threshold to improve results)
    good2 = [m for m, n in raw2 if m.distance < 0.75 * n.distance]  # Experiment with 0.75 or adjust

    # Collect 3D–2D correspondences
    obj_pts, img_pts = [], []
    for m in good2:
        if m.trainIdx in kp1_to_3d:
            obj_pts.append(X3D[kp1_to_3d[m.trainIdx]])
            img_pts.append(kp2[m.queryIdx].pt)

    obj_pts = np.array(obj_pts, dtype=np.float32)
    img_pts = np.array(img_pts, dtype=np.float32)

    if len(obj_pts) < 6:
        skipped_count += 1
        print(f"{img_name}: only {len(obj_pts)} 3D–2D pairs, skipping")
        continue

    # 3) Solve PnP with RANSAC
    ok, rvec, tvec, inliers = cv2.solvePnPRansac(
        obj_pts, img_pts, K, None,
        reprojectionError=6.0,  # Relaxed reprojection error (testing lower error)
        confidence=0.99,
        flags=cv2.SOLVEPNP_UPNP  # Switch to UPNP for better robustness
    )
    
    if not ok:
        failed_count += 1
        print(f"{img_name}: PnP failed")
        continue

    # 4) Convert rotation vector → matrix
    R, _ = cv2.Rodrigues(rvec)

    print(f"{img_name}:")
    print(" R =\n", R)
    print(" t =", tvec.ravel(), "\n")

    R_all.append(R)
    t_all.append(tvec)
    working_count += 1

# Print final stats
print(f"Stats:")
print(f"Skipped: {skipped_count}")
print(f"Failed: {failed_count}")
print(f"Working: {working_count}")

dino0047.png: only 2 3D–2D pairs, skipping
dino0046.png: only 1 3D–2D pairs, skipping
dino0045.png: only 1 3D–2D pairs, skipping
dino0044.png: only 0 3D–2D pairs, skipping
dino0043.png: only 0 3D–2D pairs, skipping
dino0042.png: only 3 3D–2D pairs, skipping
dino0041.png: only 4 3D–2D pairs, skipping
dino0040.png: only 3 3D–2D pairs, skipping
dino0039.png: only 2 3D–2D pairs, skipping
dino0038.png: PnP failed
dino0037.png: PnP failed
dino0036.png:
 R =
 [[-0.99664295 -0.05521609  0.06044845]
 [ 0.05331393 -0.99804416 -0.03264175]
 [ 0.06213257 -0.02930942  0.99763746]]
 t = [ -3.85385632  -0.48060446 -23.16041692] 

dino0035.png:
 R =
 [[-0.99735408  0.05655303  0.04567927]
 [-0.06186083 -0.99026905 -0.12466131]
 [ 0.03818479 -0.12715722  0.9911473 ]]
 t = [ -3.35766887   1.40333756 -23.2000828 ] 

dino0034.png:
 R =
 [[-0.98948768  0.1379784   0.04331396]
 [-0.14168763 -0.98491958 -0.09928767]
 [ 0.02896121 -0.10438098  0.99411562]]
 t = [ -3.17216546   0.62900293 -23.26098848] 

dino0

In [55]:
# ── cell: Visualize 3D points as a rotating GIF ──

import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation
from IPython.display import HTML

# 1) Point to your saved file:
npy_path = r"C:\Users\hassa\Downloads\dino\dino\output\points3d.npy"  # <— adjust this!

if not os.path.isfile(npy_path):
    raise FileNotFoundError(f"Could not find points file at {npy_path}")

# 2) Load the 3D points
pts = np.load(npy_path)

# 3) Create a 3D scatter
fig = plt.figure(figsize=(6,6))
ax  = fig.add_subplot(111, projection='3d')
sc  = ax.scatter(pts[:,0], pts[:,1], pts[:,2], s=2)

# keep the box aspect equal
max_range = (pts.max(axis=0) - pts.min(axis=0)).max() / 2.0
mid       = pts.mean(axis=0)
ax.set_xlim(mid[0]-max_range, mid[0]+max_range)
ax.set_ylim(mid[1]-max_range, mid[1]+max_range)
ax.set_zlim(mid[2]-max_range, mid[2]+max_range)
ax.set_xlabel('X'); ax.set_ylabel('Y'); ax.set_zlabel('Z')

# 4) Animation callback: rotate azimuth
def update(angle):
    ax.view_init(elev=30, azim=angle)
    return sc,

# 5) Build and save the animation
ani = animation.FuncAnimation(
    fig, update,
    frames=np.linspace(0, 360, 120),
    interval=100
)
gif_path = "3d_points.gif"
ani.save(gif_path, writer='pillow', fps=10)
plt.close(fig)

# 6) Display inline
HTML(f'<img src="{gif_path}" />')


In [7]:
# If you haven’t already:
# !pip install open3d

import open3d as o3d
import numpy as np

# 1) Load your optimized points (adjust the path as needed)
pts = np.load(r"C:\Users\hassa\Downloads\dino\dino\output\points3d.npy")

# 2) Build an Open3D point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pts)
pcd.paint_uniform_color([0.75, 0.75, 0.75])  # light gray

# 3) Launch the interactive viewer
o3d.visualization.draw_geometries(
    [pcd],
    window_name="Dino 3D Point Cloud",
    width=1024,
    height=768
)


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current:

In [43]:
import os
import cv2
import csv
import numpy as np
import open3d as o3d

# --- PARAMETERS ---
IMG_DIR = '.'  # folder with your dino*.png
GOOD_FILE = 'dino_good_silhouette_images.txt'
ANG_FILE = 'dino_ang.txt'
PAR_FILE = 'dino_par.txt'

RATIO_THR = 0.7  # Lowe’s ratio test threshold
RANSAC_THR = 1.0  # RANSAC reprojection threshold (pixels)
MIN_MATCHES = 8  # minimum matches after ratio test

# --- 1. Load good‐silhouette list ---
with open(GOOD_FILE, 'r') as f:
    good_imgs = {ln.strip() for ln in f if ln.strip()}

# --- 2. Load angles and keep only good images ---
ang_map = {}
with open(ANG_FILE, 'r') as f:
    for ln in f:
        parts = ln.strip().split()
        if len(parts) != 3:
            continue
        try:
            lat, lon = float(parts[0]), float(parts[1])
        except ValueError:
            continue
        name = parts[2]
        if name in good_imgs:
            ang_map[name] = (lat, lon)

# sort by latitude, then longitude
sorted_imgs = sorted(ang_map.keys(), key=lambda n: (ang_map[n][0], ang_map[n][1]))

# --- 3. Parse calibration file to extract K ---
K = None
with open(PAR_FILE, 'r') as f:
    for ln in f:
        parts = ln.strip().split()
        if len(parts) >= 7 and parts[0].endswith('.png'):
            fx, cx = float(parts[1]), float(parts[3])
            fy, cy = float(parts[5]), float(parts[6])
            K = np.array([[fx,   0, cx],
                          [  0, fy, cy],
                          [  0,   0,  1]])
            break
if K is None:
    raise RuntimeError(f"Could not parse intrinsics from {PAR_FILE}")

# prepare SIFT or ORB and BFMatcher
sift = cv2.SIFT_create()  # ORB can be used instead: sift = cv2.ORB_create()
bf = cv2.BFMatcher(cv2.NORM_L2)

# make output folders
os.makedirs('matches', exist_ok=True)

# CSV log
with open('pair_results.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['img1', 'img2', 'raw_matches', 'inliers', 't_x', 't_y', 't_z'])

    # --- 4. Loop over consecutive pairs ---
    for i in range(len(sorted_imgs) - 1):
        img1_name = sorted_imgs[i]
        img2_name = sorted_imgs[i+1]

        # load grayscale
        img1 = cv2.imread(os.path.join(IMG_DIR, img1_name), cv2.IMREAD_GRAYSCALE)
        img2 = cv2.imread(os.path.join(IMG_DIR, img2_name), cv2.IMREAD_GRAYSCALE)
        if img1 is None or img2 is None:
            print(f"Warning: couldn’t load {img1_name} or {img2_name}, skipping")
            continue

        # detect & describe
        kp1, des1 = sift.detectAndCompute(img1, None)
        kp2, des2 = sift.detectAndCompute(img2, None)

        # raw knn matches + Lowe’s ratio test
        raw = bf.knnMatch(des1, des2, k=2)
        good = []
        pts1, pts2 = [], []
        for m, n in raw:
            if m.distance < RATIO_THR * n.distance:
                good.append(m)
                pts1.append(kp1[m.queryIdx].pt)
                pts2.append(kp2[m.trainIdx].pt)

        pts1 = np.float32(pts1)
        pts2 = np.float32(pts2)
        raw_count = len(good)
        if raw_count < MIN_MATCHES:
            print(f"{img1_name}–{img2_name}: only {raw_count} after ratio test; skipping")
            continue

        # estimate Fundamental matrix via RANSAC
        F, mask = cv2.findFundamentalMat(pts1, pts2,
                                         cv2.FM_RANSAC,
                                         ransacReprojThreshold=RANSAC_THR,
                                         confidence=0.99)
        if F is None:
            print(f"{img1_name}–{img2_name}: Fundamental matrix failed; skipping")
            continue

        inlier_mask = mask.ravel().astype(bool)
        inlier_count = int(inlier_mask.sum())

        # --- 5. Compute Essential & recover pose ---
        E = K.T @ F @ K
        pts1_in = pts1[inlier_mask]
        pts2_in = pts2[inlier_mask]
        _, R, t, _ = cv2.recoverPose(E, pts1_in, pts2_in, K)

        # --- 6. Draw & save inlier matches ---
        mask_ints = [int(m) for m in inlier_mask]
        vis = cv2.drawMatches(
            img1, kp1, img2, kp2, good, None,
            matchColor=(0,255,0),
            singlePointColor=None,
            matchesMask=mask_ints,
            flags=cv2.DrawMatchesFlags_DEFAULT,
            matchesThickness=1
        )
        out_fn = f"matches/{img1_name[:-4]}_{img2_name[:-4]}_inliers.png"
        cv2.imwrite(out_fn, vis)

        # log to CSV
        tx, ty, tz = t.ravel()
        writer.writerow([img1_name, img2_name,
                         raw_count, inlier_count,
                         tx, ty, tz])

        print(f"{img1_name}–{img2_name}: {inlier_count}/{raw_count} inliers, t = [{tx:.3f}, {ty:.3f}, {tz:.3f}]")

    print("Done. Inlier visuals in ‘matches/’, summary in ‘pair_results.csv’.")

dino0049.png–dino0048.png: 61/83 inliers, t = [0.020, -0.471, -0.882]
dino0048.png–dino0047.png: 54/82 inliers, t = [0.059, -0.864, 0.500]
dino0047.png–dino0046.png: 45/77 inliers, t = [-0.005, -0.534, -0.846]
dino0046.png–dino0045.png: 34/55 inliers, t = [0.035, -0.339, 0.940]
dino0045.png–dino0044.png: 25/45 inliers, t = [0.048, 0.091, 0.995]
dino0044.png–dino0043.png: 21/33 inliers, t = [-0.064, 0.381, 0.922]
dino0043.png–dino0042.png: 13/22 inliers, t = [-0.002, -0.036, 0.999]
dino0042.png–dino0041.png: 14/20 inliers, t = [-0.050, 0.039, 0.998]
dino0041.png–dino0040.png: 14/23 inliers, t = [0.064, -0.224, -0.972]
dino0040.png–dino0039.png: 21/34 inliers, t = [0.041, -0.464, -0.885]
dino0039.png–dino0038.png: 26/47 inliers, t = [0.032, -0.220, -0.975]
dino0038.png–dino0037.png: 45/74 inliers, t = [0.020, -0.414, -0.910]
dino0037.png–dino0036.png: 53/81 inliers, t = [0.024, -0.326, -0.945]
dino0036.png–dino0035.png: 71/104 inliers, t = [0.019, -0.433, -0.901]
dino0035.png–dino0034.pn

In [44]:
import numpy as np
import cv2

# Assume these variables are defined already:
# K           : (3x3) calibration matrix
# F           : (3x3) fundamental matrix
# pts1, pts2  : (N×2) all matched points before RANSAC
# mask        : (N×1) inlier mask from findFundamentalMat

# 1) Extract only the inlier correspondences
mask_bool = mask.ravel().astype(bool)
pts1_in = pts1[mask_bool]
pts2_in = pts2[mask_bool]

# 2) Compute and fix the Essential matrix
E = K.T @ F @ K  # Essential matrix from calibration matrix and fundamental matrix
U, S, Vt = np.linalg.svd(E)

# Enforce singularity constraint (σ, σ, 0) on the Essential matrix
sigma = (S[0] + S[1]) / 2.0
E = U @ np.diag([sigma, sigma, 0]) @ Vt

# 3) Compute the four possible camera poses (R, t)
W = np.array([[0, -1,  0],
              [1,  0,  0],
              [0,  0,  1]])

# Create possible rotation matrices
R1 = U @ W @ Vt
R2 = U @ W.T @ Vt
t0 = U[:, 2]

# Ensure det(R) = +1 for valid rotation matrices
if np.linalg.det(R1) < 0: R1 *= -1
if np.linalg.det(R2) < 0: R2 *= -1

# List of 4 possible (R, t) hypotheses
candidates = [
    (R1,  t0),
    (R1, -t0),
    (R2,  t0),
    (R2, -t0),
]

# Helper function: triangulate points and count how many points have positive depth
def count_in_front(R, t, K, pts1, pts2):
    # Projection matrices for both cameras
    P1 = K @ np.hstack((np.eye(3), np.zeros((3, 1))))  # Camera 1: identity matrix for R, zero for t
    P2 = K @ np.hstack((R, t.reshape(3, 1)))  # Camera 2: [R|t] matrix
    
    # Convert points to normalized coordinates (undistorted)
    p1n = cv2.undistortPoints(pts1.reshape(-1, 1, 2), K, None)
    p2n = cv2.undistortPoints(pts2.reshape(-1, 1, 2), K, None)
    
    # Triangulate points (4xN homogeneous coordinates)
    X4 = cv2.triangulatePoints(P1, P2, p1n, p2n)
    X3 = (X4[:3] / X4[3]).T  # Convert to Euclidean coordinates (N×3)
    
    # Check depths in both camera frames
    z1 = X3[:, 2]  # Depth in the first camera (positive is in front of the camera)
    z2 = (R @ X3.T + t.reshape(3, 1))[2]  # Depth in the second camera
    
    # Count how many points have positive depth in both cameras
    return np.sum((z1 > 0) & (z2 > 0)), X3

# 4) Select the best pose based on the count of valid points
best_count = 0
for R_c, t_c in candidates:
    count, X3 = count_in_front(R_c, t_c, K, pts1_in, pts2_in)
    if count > best_count:
        best_count = count
        R_final, t_final, X_final = R_c, t_c, X3

print(f"→ Best pose has {best_count} points in front.")
print("Rotation (R_final):\n", R_final)
print("Translation (t_final):\n", t_final)

# 5) (optional) Convert 3D points to Euclidean coordinates
X3D = X_final  # Shape: (best_count x 3)
print("First few 3D points:\n", X3D[:5])

→ Best pose has 84 points in front.
Rotation (R_final):
 [[ 0.99672795 -0.08008046  0.01097805]
 [ 0.07973585  0.99639762  0.02887849]
 [-0.01325111 -0.02790866  0.99952264]]
Translation (t_final):
 [-0.00692055 -0.49965152 -0.86619886]
First few 3D points:
 [[-2.376378  -1.3716363 23.33098  ]
 [-2.376358  -1.3714566 23.331608 ]
 [-2.3762972 -1.3714077 23.331572 ]
 [-2.3762898 -1.3713775 23.3317   ]
 [-2.3761408 -1.3717183 23.330488 ]]


In [45]:
import os
import cv2
import numpy as np

# --- 0) Reconstruct inlier_matches from the initial pair ---
mask_bool = mask.ravel().astype(bool)
inlier_matches = [m for m, valid in zip(good, mask_bool) if valid]

# --- 1) Convert homogeneous 4D points → Euclidean 3D ---
X3D = (X4_final[:3] / X4_final[3]).T   # shape: (M, 3)

# --- 2) Build a map: kp1 index → index in X3D ---
kp1_to_3d = { m.queryIdx: i for i, m in enumerate(inlier_matches) }

# Prepare storage for rotation and translation results
R_all, t_all = [], []

# --- 3) Loop over new images to perform incremental registration ---
for img_name in sorted_imgs[2:]:
    # Load new image & compute features
    img = cv2.imread(os.path.join(IMG_DIR, img_name), cv2.IMREAD_GRAYSCALE)
    kp2, des2 = sift.detectAndCompute(img, None)

    # Match back to the first image's descriptors using BFMatcher
    raw2 = bf.knnMatch(des2, des1, k=2)
    
    # Apply Lowe's ratio test to filter out bad matches
    good2 = [m for m, n in raw2 if m.distance < RATIO_THR * n.distance]

    # Collect 3D–2D correspondences
    obj_pts, img_pts = [], []
    for m in good2:
        if m.trainIdx in kp1_to_3d:
            obj_pts.append(X3D[kp1_to_3d[m.trainIdx]])
            img_pts.append(kp2[m.queryIdx].pt)

    obj_pts = np.array(obj_pts, dtype=np.float32)
    img_pts = np.array(img_pts, dtype=np.float32)

    # Skip frames with insufficient correspondences
    if len(obj_pts) < 6:
        print(f"{img_name}: only {len(obj_pts)} 3D–2D pairs, skipping")
        continue

    # --- 4) Solve PnP with RANSAC ---
    # Improve RANSAC parameters and reprojection error threshold for better accuracy
    success, rvec, tvec, inliers = cv2.solvePnPRansac(
        obj_pts, img_pts, K, None,
        reprojectionError=4.0,  # Reduced reprojection error threshold for better accuracy
        confidence=0.99,
        flags=cv2.SOLVEPNP_ITERATIVE
    )

    if not success:
        print(f"{img_name}: PnP failed")
        continue

    # --- 5) Convert rotation vector to matrix ---
    R, _ = cv2.Rodrigues(rvec)

    print(f"{img_name}:")
    print(" R =\n", R)
    print(" t =", tvec.ravel(), "\n")

    R_all.append(R)
    t_all.append(tvec)

# --- 6) Optional: Refine results (Bundle Adjustment or Pose Graph Optimization) ---
# If you want to further improve results, use bundle adjustment or pose graph optimization after this loop.

dino0047.png: only 0 3D–2D pairs, skipping
dino0046.png: only 0 3D–2D pairs, skipping
dino0045.png: only 1 3D–2D pairs, skipping
dino0044.png: only 0 3D–2D pairs, skipping
dino0043.png: only 0 3D–2D pairs, skipping
dino0042.png: only 2 3D–2D pairs, skipping
dino0041.png: only 2 3D–2D pairs, skipping
dino0040.png: only 3 3D–2D pairs, skipping
dino0039.png: only 0 3D–2D pairs, skipping
dino0038.png: only 3 3D–2D pairs, skipping
dino0037.png: PnP failed
dino0036.png: PnP failed
dino0035.png: PnP failed
dino0034.png: PnP failed
dino0033.png: PnP failed
dino0032.png: PnP failed
dino0031.png: PnP failed
dino0030.png: PnP failed
dino0006.png: only 1 3D–2D pairs, skipping
dino0005.png: only 0 3D–2D pairs, skipping
dino0004.png: only 1 3D–2D pairs, skipping
dino0003.png: only 0 3D–2D pairs, skipping
dino0002.png: only 1 3D–2D pairs, skipping
dino0001.png: only 1 3D–2D pairs, skipping
dino0050.png: only 1 3D–2D pairs, skipping
dino0051.png: only 0 3D–2D pairs, skipping
dino0052.png: only 0 3D–2D

In [48]:
import os
import cv2
import numpy as np

# Initialize FLANN-based matcher for better feature matching
FLANN_INDEX_KDTREE = 1  # Flann index for KDTREE algorithm
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)  # or pass empty dictionary for default

# Initialize SIFT detector
sift = cv2.SIFT_create()

# Initialize FLANN-based matcher
flann = cv2.FlannBasedMatcher(index_params, search_params)

# 0) Reconstruct inlier_matches from your initial pair
mask_bool = mask.ravel().astype(bool)
inlier_matches = [m for m, valid in zip(good, mask_bool) if valid]

# 1) Convert homogeneous 4D points → Euclidean 3D
X3D = (X4_final[:3] / X4_final[3]).T   # shape: (M,3)

# 2) Build a map: kp1 index → index in X3D
kp1_to_3d = { m.queryIdx: i for i, m in enumerate(inlier_matches) }

# Prepare storage for rotation and translation results
R_all, t_all = [], []

# 3) Loop over new images to perform incremental registration
for img_name in sorted_imgs[2:]:
    # Load new image & compute features
    img = cv2.imread(os.path.join(IMG_DIR, img_name), cv2.IMREAD_GRAYSCALE)

    # Check if the image is loaded successfully
    if img is None:
        print(f"{img_name}: Failed to load image.")
        continue

    kp2, des2 = sift.detectAndCompute(img, None)
    
    # Make sure descriptors are not None before proceeding
    if des2 is None:
        print(f"{img_name}: No keypoints or descriptors found.")
        continue

    # Match back to the first image's descriptors using FLANN-based matcher
    # Check if des1 (the descriptors from the reference image) are also not None
    if des1 is None:
        print(f"Reference image descriptors (des1) are empty. Skipping {img_name}.")
        continue

    # Perform FLANN-based matching with k=2 (for Lowe's ratio test)
    raw2 = flann.knnMatch(des2, des1, k=2)

    # Apply Lowe's ratio test to filter out bad matches (adjust threshold if necessary)
    good2 = [m for m, n in raw2 if m.distance < 0.6 * n.distance]  # use 0.6 threshold for stricter matching

    # Collect 3D–2D correspondences
    obj_pts, img_pts = [], []
    for m in good2:
        if m.trainIdx in kp1_to_3d:
            obj_pts.append(X3D[kp1_to_3d[m.trainIdx]])
            img_pts.append(kp2[m.queryIdx].pt)

    obj_pts = np.array(obj_pts, dtype=np.float32)
    img_pts = np.array(img_pts, dtype=np.float32)

    # Skip frames with insufficient correspondences
    if len(obj_pts) < 6:
        print(f"{img_name}: only {len(obj_pts)} 3D–2D pairs, skipping")
        continue

    # 4) Solve PnP with RANSAC
    success, rvec, tvec, inliers = cv2.solvePnPRansac(
        obj_pts, img_pts, K, None,
        reprojectionError=4.0,  # Lower reprojection error for higher accuracy
        confidence=0.99,
        flags=cv2.SOLVEPNP_ITERATIVE
    )

    if not success:
        print(f"{img_name}: PnP failed")
        continue

    # 5) Convert rotation vector to matrix
    R, _ = cv2.Rodrigues(rvec)

    print(f"{img_name}:")
    print(" R =\n", R)
    print(" t =", tvec.ravel(), "\n")

    R_all.append(R)
    t_all.append(tvec)

# Optionally, use Bundle Adjustment or Pose Graph Optimization to refine results after the loop
# For better results, bundle adjustment can optimize both 3D points and camera poses.

dino0047.png: only 0 3D–2D pairs, skipping
dino0046.png: only 0 3D–2D pairs, skipping
dino0045.png: only 1 3D–2D pairs, skipping
dino0044.png: only 0 3D–2D pairs, skipping
dino0043.png: only 0 3D–2D pairs, skipping
dino0042.png: only 0 3D–2D pairs, skipping
dino0041.png: only 0 3D–2D pairs, skipping
dino0040.png: only 2 3D–2D pairs, skipping
dino0039.png: only 0 3D–2D pairs, skipping
dino0038.png: only 0 3D–2D pairs, skipping
dino0037.png: only 1 3D–2D pairs, skipping
dino0036.png: only 3 3D–2D pairs, skipping
dino0035.png: PnP failed
dino0034.png: PnP failed
dino0033.png: PnP failed
dino0032.png: PnP failed
dino0031.png: PnP failed
dino0030.png: only 3 3D–2D pairs, skipping
dino0006.png: only 0 3D–2D pairs, skipping
dino0005.png: only 0 3D–2D pairs, skipping
dino0004.png: only 0 3D–2D pairs, skipping
dino0003.png: only 0 3D–2D pairs, skipping
dino0002.png: only 1 3D–2D pairs, skipping
dino0001.png: only 0 3D–2D pairs, skipping
dino0050.png: only 0 3D–2D pairs, skipping
dino0051.png: o